In [ ]:
from flask import *
from langchain import OpenAI,PromptTemplate,LLMChain

import csv
import openai

In [ ]:
#api 키 불러오기
def load_api_key(file_path):
    with open(file_path, 'r') as file:
        api_key = file.read().strip()
    return api_key

# OpenAI API 설정
api_key = load_api_key("") #api 파일 경로 설정
model = OpenAI(openai_api_key=api_key, model="GPT-4o")

# LangChain 고민 여부 설정
template = PromptTemplate(
    template="사용자의 입력은 : '{input}'\n고민, 걱정, 또는 문제를 표현하고 있는지? '예', '아니오','맞다'등 명확하고 간결하게 표현",
    input_variables=["input"]
)
is_problem_chain = LLMChain(llm=model, prompt_template=template)

# CSV 파일 로드
def load_response_csv(csv_file):
    responses = {}
    with open(csv_file, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        for row in reader:
            reason = row["고민이유"]  # 고민의 카테고리명
            response = row["답변"]    # 답변
            responses[reason] = response
    return responses

# CSV 파일 로드
responses = load_response_csv("파일경로.csv")  # 파일 경로 설정

def process_input(user_nick,user_input):
    try:
        # GPT로 고민 여부 판단
        is_problem_response = is_problem_chain.run(input=user_input).strip()
        # 긍정표현
        pos_res = ["예", "네", "맞다", "맞습니다", "그렇다",
            "그렇습니다", "고민이라고 판단된다", "고민입니다", "예, 고민이다"]
        
        is_problem_flag = int(is_problem_response in pos_res)

        # 고민이라고 판단한 경우
        if is_problem_flag:
            reason_template = PromptTemplate(
                template=("입력값은 : '{user_input}'\n"
                          "고민이라고 판단한 이유는? (예: )\n예시중에 선택"),#csv파일의 카테고리 작성할것
                input_variables=["input"]
            )
            reason_chain = LLMChain(llm=model, prompt_template=reason_template)
            reason_response = reason_chain.run(input=user_input)
            reason = reason_response.strip()  # 고민 이유는 카테고리명

            # 고민 이유에 맞는 답변 찾기
            if reason in responses:
                answer = responses[reason]
            else:
                answer = responses.get("기타")  # '기타' 카테고리 참조, 없으면 기본 답변 제공

            # 답변 길이 제한
            if len(answer) > 500:
                answer = answer[:500]  # 500자 이내로 자르기

            return jsonify({
                "user" : user_nick,
                "input": user_input,
                "isProblem": is_problem_flag,
                "reason": reason,
                "answer": answer
            })
        else:
            # 고민이 아닌 경우 일반적인 GPT 응답
            general_template = PromptTemplate(
                template="사용자의 입력: '{user_input}'\n 일반적인 답변을 해주세요.)",
                input_variables=["input"]
            )
            general_chain = LLMChain(llm=model, prompt_template=general_template)
            general_response = general_chain.run(input=user_input)
            answer = general_response.strip()

            # 답변 길이 제한
            if len(answer) > 500:
                answer = answer[:500]  # 500자 이내로 자르기

            return jsonify({
                "user_nick" : user_nick,
                "answer": answer
            })
    except Exception as e:
        print("Error processing input:", e)
        return jsonify({"error": "Error processing input"}), 500